In [1]:
from dotenv import load_dotenv
import os
import pinecone
from pinecone import Pinecone as PineconeClient
from pinecone import ServerlessSpec

In [2]:
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pinecone_environment = os.getenv("PINECONE_ENVIRONMENT")

In [3]:
pc = PineconeClient(api_key=pinecone_api_key)

In [6]:
index_name = "movie-index"
if not pc.has_index(index_name):
  pc.create_index(
    index_name,
    dimension=1536,
    spec=ServerlessSpec(
      cloud="aws",
      region="us-east-1"
    )
  )
# 생성된 인덱스 연결
index = pc.Index(index_name)

c:\Users\user\Downloads\Data_A\prompt_basic\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
data = [
  {
      "title": "응답하라 1988",
      "year": 2015,
      "genre": ["드라마", "코미디"],
      "director": "신원호",
      "actors": ["혜리", "박보검", "류준열"],
      "rating": 9.2,
      "synopsis": "1988년 서울, 쌍문동 이웃들 사이의 우정과 가족애를 그린 드라마.",
  },
  {
      "title": "기생충",
      "year": 2019,
      "genre": ["드라마", "스릴러"],
      "director": "봉준호",
      "actors": ["송강호", "이선균", "조여정"],
      "rating": 8.6,
      "synopsis": "가난한 가족과 부유한 가족 사이 벌어지는 블랙코미디 풍자의 스릴러 영화.",
  },
]

In [8]:
!uv add pandas

Resolved 87 packages in 618ms
Installed 3 packages in 1.01s
 + pandas==2.3.0
 + pytz==2025.2
 + tzdata==2025.2


In [9]:
import pandas as pd
df = pd.DataFrame(data)
df.head(2)

,title,year,genre,director,actors,rating,synopsis
0,응답하라 1988,2015,"[드라마, 코미디]",신원호,"[혜리, 박보검, 류준열]",9.2,"1988년 서울, 쌍문동 이웃들 사이의 우정과 가족애를 그린 드라마."
1,기생충,2019,"[드라마, 스릴러]",봉준호,"[송강호, 이선균, 조여정]",8.6,가난한 가족과 부유한 가족 사이 벌어지는 블랙코미디 풍자의 스릴러 영화.


In [10]:
texts = df["synopsis"].tolist()
texts

['1988년 서울, 쌍문동 이웃들 사이의 우정과 가족애를 그린 드라마.',
 '가난한 가족과 부유한 가족 사이 벌어지는 블랙코미디 풍자의 스릴러 영화.']

"어떤 필트로 검색 조건을 걸 것인가?"를 미리 생각하여 그 필드를 메타데이터로 저장  
"벡터 쿼리" + "메타데이터 필터" 조합으로 강력한 검색을 구현  
Pinecone에 벡터를 올릴 때, 함께 저장할 메타데이터(metadata)를 잘 구조화하는 것이 중요합니다. 메타데이터는 각 벡터의 부가정보로 저장되며, 이후 검색 시 필터(filter)로 활용할 수 있습니다. 이번 영화 데이터 예시에서는 다음과 같은 메타데이터 필드를 설계할 수 있습니다.  

title(제목) - 영화 또는 드라마 제목(문자열)  
year(개봉년도) - 작품의 연도(정수)  
genre(장르) - 작품의 주요 장르(문자열 혹은 문자열의 리스트)  
director(감독) - 감독 이름(문자열)  
actors(출연 배우) - 주요 배우들의 이름(여러 명일 경우 문자열 리스트)  
rating(평점) - 작품의 평점(실수 또는 정수)  
synopsis(줄거리) - 작품의 간략한 줄거리 설명(문자열, 한국어)  
위의 필드 중 synopsis는 검색을 위한 본문 텍스트이며, 나머지 필드는 작품을 속성별로 필터링하거나 결과를 보여줄 때 활용합니다.  

In [14]:
from openai import OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
response = client.embeddings.create(input=texts, model="text-embedding-3-small")
embeddings = [item.embedding for item in response.data]

In [16]:
records_to_upsert = []
for idx, row in df.iterrows():
  vec_id = f"movie-{idx}"
  vec = embeddings[idx]
  meta = {
    "title": row["title"],
    "year": row["year"],
    "genre": row["genre"],
    "director": row["director"],
    "actors": row["actors"],
    "rating": row["rating"],
    "synopsis": row["synopsis"],
  }
records_to_upsert.append((vec_id,vec, meta))

In [18]:
index.upsert(vectors=records_to_upsert)

{'upserted_count': 1}

In [19]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 1}},
 'total_vector_count': 1,
 'vector_type': 'dense'}